# [SOURCE] OVERALL ACCURACY

In [1]:
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.append("./scripts/modules")

from benchmarking_definitions import *

In [2]:
tagsRv = pd.read_csv("./data/source/FINAL_tag_list.csv", names=["tag"])["tag"].values
tagsL1234 = pd.read_csv("./data/source/FINAL_tag_list_L1234.csv", names=["tag"]).tag.values

len(tagsRv), len(tagsL1234)

(2500, 1000)

In [3]:
ISS_Rv_mutant_summary_source = "/n/scratch/users/s/sm624/benchmarking/variant_summaries/ISS_H37Rv"
ISS_L1234_mutant_summary_source = "/n/scratch/users/s/sm624/benchmarking/variant_summaries/ISS_L1234"

# [done] ISS H37Rv weighted F1

scaling factor: median of mutation count in region/total of all mutation counts

DR + other = 0.0 + 106<br>
LM = 4.0<br>
HT = 1.0

**Total = 111**

In [4]:
scaling_factor = {"DR": 106, "HT": 1, "LM": 4}
scaling_factor_total = sum(scaling_factor.values())

In [5]:
Rv_weighted_F1_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "tool", "weighted_F1"])
df_i = 0

count = 0
for tag in tagsRv:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ISS_Rv_mutant_summary_source, tag), low_memory=False)
        
    for tool in tools:
        
        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FP = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 0)]

        precision_values, recall_values = {}, {}
        
        for region in regions:

            region_TP = tool_TP[tool_TP.REGION_sim == region]
            region_FN = tool_FN[tool_FN.REGION_sim == region]
            region_FP = tool_FP[tool_FP.REGION_sim == region]

            num_region_TP = region_TP.shape[0]
            num_region_FN = region_FN.shape[0]
            num_region_FP = region_FP.shape[0]

            region_precision = calc_precision(num_region_TP, num_region_FP)
            region_recall = calc_recall(num_region_TP, num_region_FN)

            precision_values[region] = region_precision
            recall_values[region] = region_recall

        weighted_precision = sum([precision_values[r]*scaling_factor[r] for r in regions])/scaling_factor_total
        weighted_recall = sum([recall_values[r]*scaling_factor[r] for r in regions])/scaling_factor_total
        
        weighted_F1 = calc_F1(weighted_precision, weighted_recall)
            
        Rv_weighted_F1_df.loc[df_i] = [tag, simulation, depth, freq, tool, weighted_F1]
        df_i += 1
        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [6]:
Rv_weighted_F1_df.head()

,tag,simulation,depth,freq,tool,weighted_F1
0,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,FB,0.0
1,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,LF,0.0
2,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,MT,0.0
3,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,PL,0.0
4,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,VD,0.0


In [7]:
Rv_weighted_F1_df.to_csv("./data/overall_accuracy/ISS_H37Rv_weighted_F1.csv", index=False)

# [done] ISS average precision & recall by region

## H37Rv

In [9]:
Rv_precision_recall_by_region_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "genome", "tool", "region", "precision", "recall"])
df_i = 0

count = 0
for tag in tagsRv:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ISS_Rv_mutant_summary_source, tag), low_memory=False)
        
    for tool in tools:
        
        tool_TP = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FP = mutant_summary_df[(mutant_summary_df.GT == 0) & (mutant_summary_df["{}_found".format(tool)] == 1)]
        tool_FN = mutant_summary_df[(mutant_summary_df.GT == 1) & (mutant_summary_df["{}_found".format(tool)] == 0)]

        for region in regions:

            region_TP = tool_TP[tool_TP.REGION_sim == region]
            region_FN = tool_FN[tool_FN.REGION_sim == region]
            region_FP = tool_FP[tool_FP.REGION_sim == region]

            num_region_TP = region_TP.shape[0]
            num_region_FN = region_FN.shape[0]
            num_region_FP = region_FP.shape[0]

            region_precision = calc_precision(num_region_TP, num_region_FP)
            region_recall = calc_recall(num_region_TP, num_region_FN)

            Rv_precision_recall_by_region_df.loc[df_i] = [tag, simulation, depth, freq, "H37Rv", tool, region, region_precision, region_recall]
            df_i += 1
                                        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [10]:
Rv_precision_recall_by_region_df.head()

,tag,simulation,depth,freq,genome,tool,region,precision,recall
0,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,DR,0.0,0.0
1,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,HT,0.0,0.0
2,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,FB,LM,0.0,0.0
3,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,LF,DR,0.0,0.0
4,sim1_h37rv_mutant_1_50_0.01,1,50,0.01,H37Rv,LF,HT,0.0,0.0


In [11]:
Rv_precision_recall_by_region_df.to_csv("./data/overall_accuracy/ISS_H37Rv_precision_recall_by_region.csv", index=False)

## L1234 (DR only)

In [8]:
region = "DR"

L1234_DR_precision_recall_df = pd.DataFrame(columns=["tag", "simulation", "depth", "freq", "genome", "lineage", "tool", "region", "precision", "recall"])
df_i = 0

count = 0
for tag in tagsL1234:
    
    if count % 10 == 0:
        print(count, end=" ")
    count += 1
    
    simulation = int(tag.split("_")[0].strip("sim"))
    lineage = int(tag.split("_")[1].strip("L"))
    depth = int(tag.split("_")[-2])
    freq = float(tag.split("_")[-1])
    
    mutant_summary_df = pd.read_csv("{0}/{1}.variant_summary.csv".format(ISS_L1234_mutant_summary_source, tag), low_memory=False)
    mutant_summary_nonbaseline_df = mutant_summary_df[mutant_summary_df.L_GT == 0]
    DR_mutant_summary_df = mutant_summary_nonbaseline_df[mutant_summary_nonbaseline_df.REGION_sim == region]
        
    for tool in tools:
        
        DR_tool_TP = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 1) & (DR_mutant_summary_df["{}_found".format(tool)] == 1)]
        DR_tool_FN = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 1) & (DR_mutant_summary_df["{}_found".format(tool)] == 0)]
        DR_tool_FP = DR_mutant_summary_df[(DR_mutant_summary_df.GT == 0) & (DR_mutant_summary_df["{}_found".format(tool)] == 1)]
        DR_tool_FP = DR_tool_FP[DR_tool_FP["{}_AF".format(tool)] <= 0.50]

        num_DR_TP = DR_tool_TP.shape[0]
        num_DR_FN = DR_tool_FN.shape[0]
        num_DR_FP = DR_tool_FP.shape[0]

        DR_precision = calc_precision(num_DR_TP, num_DR_FP)
        DR_recall = calc_recall(num_DR_TP, num_DR_FN)

        L1234_DR_precision_recall_df.loc[df_i] = [tag, simulation, depth, freq, "L1234", lineage, tool, region, DR_precision, DR_recall]
        df_i += 1
                                        

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 

In [9]:
L1234_DR_precision_recall_df.head()

,tag,simulation,depth,freq,genome,lineage,tool,region,precision,recall
0,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,FB,DR,0.0,0.0
1,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,LF,DR,0.0,0.0
2,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,MT,DR,0.0,0.0
3,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,PL,DR,0.0,0.0
4,sim1_L1_mutant_50_0.01,1,50,0.01,L1234,1,VD,DR,0.0,0.0


In [10]:
L1234_DR_precision_recall_df.to_csv("./data/overall_accuracy/ISS_L1234_precision_recall_DR.csv", index=False)